In [ ]:
# Install necessary libraries
%pip install opencv-python
%pip install numpy
%pip install ultralytics
%pip install lap
%pip install shapely

In [5]:
# Import required libraries
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO
from shapely.geometry import LineString, Point

In [ ]:
# Load a pre-trained YOLO model for object detection
model = YOLO('model/yolov8customv2.pt', task="detect")

# Set the path to the input video file
video_path = "videos/Traffic_3.mp4"
video = cv2.VideoCapture(video_path)  # Initialize video capture object

# Dictionary to store the history of object tracks
track_history = defaultdict(lambda: [])

# Define the points for a counting line
line_points = [(1200, 465), (100, 465)]  # Points for a horizontal line

# List of class IDs to count (e.g., cars, motorcycles, buses, trucks)
classes_to_count = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

# Define the counting line as a Shapely LineString
counting_line = LineString(line_points)

# Define y-coordinates for additional lines (used for visualization or further analysis)
y1 = 420
y2 = 510

# Initialize counters for vehicles going in different directions
south_counts = 0
north_counts = 0
counting_list = []  # List to track counted vehicle IDs

# Loop through frames of the video
while video.isOpened():
    success, frame = video.read()  # Read a frame from the video

    if success:
        # Perform object tracking using the YOLO model
        results = model.track(frame, persist=True, classes=classes_to_count, show=False)

        # Calculate half of the frame width
        half_width = frame.shape[1] // 2

        # Define points for different lines on the frame
        line_points_left1 = [(0, y1), (half_width, y1)]  # Left line 1 (color: red)
        line_points_left2 = [(half_width, y1), (frame.shape[1], y1)]  # Left line 2 (color: blue)
        line_points_right1 = [(0, y2), (half_width, y2)]  # Right line 1 (color: green)
        line_points_right2 = [(half_width, y2), (frame.shape[1], y2)]  # Right line 2 (color: yellow)

        # Get detected bounding boxes and their respective track IDs
        boxes = results[0].boxes.xywh.cpu()  # Convert to CPU
        track_ids = results[0].boxes.id.int().cpu().tolist()  # Convert to list

        # Visualize the detection results on the frame
        frame = results[0].plot()

        # Iterate through each detected object
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box  # Get bounding box coordinates
            track = track_history[track_id]  # Retrieve the tracking history for the object
            track.append((float(x), float(y)))  # Append the current position to the track history
            if len(track) > 5:  # Limit the tracking history to the last 5 points
                track.pop(0)

            # Calculate speed and direction if the object has been tracked for at least two frames
            if len(track) >= 2:
                current_pos = np.array(track[-1])
                previous_pos = np.array(track[-2])
                speed = np.linalg.norm(current_pos - previous_pos)  # Calculate speed

                # Calculate the direction based on the bounding box contour
                contour = np.array([[int(x), int(y)], [int(x + w), int(y)], [int(x + w), int(y + h)], [int(x), int(y + h)]])
                rect = cv2.minAreaRect(contour)  # Get the minimum area rectangle for the contour
                angle = rect[2]  # Get the angle of the rectangle

                # Determine direction based on vertical movement
                direction = "North" if current_pos[1] < previous_pos[1] else "South"

                # Calculate the distance to the counting line
                distance = Point(track[-1]).distance(counting_line)
                if distance < 15:  # If the object is close to the line
                    if track_id not in counting_list:  # Check if the object has already been counted
                        counting_list.append(track_id)  # Mark the object as counted
                        if box[0] < counting_line.centroid.x:
                            north_counts += 1  # Increment northbound count
                        else:
                            south_counts += 1  # Increment southbound count

                # Prepare text for vehicle counts
                south_text = 'Southbound vehicles : ' + f'{south_counts}'
                north_text = 'Northbound Vehicles : ' + f'{north_counts}'

                # Display counts on the frame
                cv2.putText(frame, south_text, (950, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
                cv2.putText(frame, north_text, (100, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

                # Display speed and direction on the frame
                cv2.putText(frame, f"Speed: {speed:.2f} px/frame", (int(x), int(y) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                cv2.putText(frame, f"Direction: {direction}", (int(x), int(y) + 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                # Draw the counting line
                cv2.polylines(frame, [np.array(line_points, dtype=np.int32)], isClosed=True, color=(255, 255, 255), thickness=1)

                # Draw the track history of the objects
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(frame, [points], isClosed=False, color=(255, 255, 255), thickness=1)
                cv2.circle(frame, (int(track[-1][0]), int(track[-1][1])), 2, (255, 255, 255), -1)

        # Display the annotated frame
        cv2.imshow("Traffic Tracking", frame)

        # Break the loop if 'ESC' is pressed
        if cv2.waitKey(1) & 0xFF == 27:  # Check for 'ESC' key (ASCII Value 27)
            break
    else:
        break

# Release the video capture object
video.release()

# Close the display window
cv2.destroyAllWindows()